## Sun's VIP
$\newcommand{\n}[1]{\left\|#1 \right\|}$ 
$\renewcommand{\a}{\alpha}             $ 
$\renewcommand{\b}{\beta}              $ 
$\renewcommand{\c}{\gamma}             $ 
$\renewcommand{\d}{\delta}             $ 
$\newcommand{\D}{\Delta}               $ 
$\newcommand{\la}{\lambda}             $ 
$\renewcommand{\t}{\tau}               $ 
$\newcommand{\s}{\sigma}               $ 
$\newcommand{\e}{\varepsilon}          $ 
$\renewcommand{\th}{\theta}            $ 
$\newcommand{\x}{\bar x}               $ 
$\newcommand{\R}{\mathbb R}            $ 
$\newcommand{\N}{\mathbb N}            $ 
$\newcommand{\Z}{\mathbb Z}            $ 
$\newcommand{\E}{\mathcal E}           $ 
$\newcommand{\lr}[1]{\left\langle #1\right\rangle}$
$\newcommand{\nf}[1]{\nabla f(#1)}     $
$\newcommand{\hx}{\hat x}               $
$\newcommand{\hy}{\hat y}               $
$\DeclareMathOperator{\prox}{prox}      $
$\DeclareMathOperator{\argmin}{argmin}  $
$\DeclareMathOperator{\dom}{dom}        $
$\DeclareMathOperator{\id}{Id}          $
$\DeclareMathOperator{\conv}{conv}      $

We study a nonlinear VI, proposed by
D.Sun
\begin{equation}
    \lr{F(x^*), x-x^*} \geq 0 \quad \forall x \in C,
\end{equation}
 where
 \begin{align*}
    F(x)   & = F_1(x) + F_2(x),\\
    F_1(x) & = (f_1(x),f_2(x),\dots,   f_d(x)),\\
    F_2(x) & = Dx+c, \\
    f_i(x) & = x_{i-1}^2 + x_i^2 +  x_{i-1}x_i + x_i x_{i+1},\quad   i=1,2,\dots, d,\\
    x_0 & = x_{d+1} = 0,
  \end{align*}
  Here $D$ is a square matrix $d\times d$ defined by condition
$$d_{ij}=
\begin{cases}
  4, & i = j,\\
  1, & i-j=1,\\
  -2,& i-j = -1,\\
  0, & \text{otherwise},
\end{cases}
$$
and $c=(-1,-1,\dots, -1)$.
We consider three different feasible set $C$: (a) $C = \R^d$, (b) $C =
[-0.25,0.25]^d$, (c) $C =\{x\in \R^d_+ \colon x_1 + \dots + x_d = d \} $


In [1]:
import matplotlib.pyplot as plt
import scipy.sparse as sr

from methods.algorithms_terminate import *
from misc.opt_operators import *

%load_ext autoreload
%matplotlib inline
%autoreload 2

In [4]:
dim = 1000

#D = 4 * np.eye(dim) + np.diag(np.ones(dim - 1), -1) + np.diag(-2 * np.ones(dim - 1), 1)
D = sr.diags([4,1,-2], [0,-1,1], shape=(dim, dim), format='csr')

def f(x):
    x2 = x**2
    s1 = x2 + np.append(0,x2[:-1])
    xy = x[:-1]*x[1:]
    xy1 = np.append(xy,0)
    xy2 = np.append(0,xy)
    s2 = xy1 + xy2
    return s1 + s2

F = lambda x:  f(x) + D.dot(x) - np.ones(dim)

J = lambda x, rho: LA.norm(x - prox_g(x - rho*F(x), rho))

# initial point
np.random.seed(123)
x0 = np.random.uniform(-10,10, dim)

We consider two cases for the feasible sets below: $C = \R^d_+$, $C = \{x\in \R^d_+ \colon x_1+\dots x_d =d\}$. Choose the respective proximal operator:

In [5]:
#prox_g = lambda x, rho: np.fmax(x,0)
prox_g = lambda x, rho: proj_simplex(x,dim)


In [6]:
N = 1000
ans1 = tseng_fbf_linesearch(F, prox_g, x0, delta=1.5, numb_iter=N)
ans2 = alg_VI_proj(F, prox_g, x0, tau_0=1, constr=True, numb_iter=N)
ans3 = alg_VI_prox(F, prox_g, x0, numb_iter=N)

---- FBF alg.----
Number of iterations: 138
Number of prox_g: 290
Number of F, : 428
Time execution: 0.07
---- Alg. 1 ----
Number of iterations: 78
Number of prox_g: 78
Number of F, : 166
Time execution: 0.02
---- Alg. 2 ----
Number of iterations: 78
Number of prox_g: 78
Number of F, : 154
Time execution: 0.02


In [7]:
plt.plot(ans1[0],'b')
plt.plot(ans2[0],'g')
plt.plot(ans3[0],'r')
plt.yscale('log')
plt.show()